In [ ]:
import pandas as pd
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from google.colab import files
uploaded = files.upload()
# Here, upload the zipped file of GurNum folder
import zipfile
import io
data =
zipfile.ZipFile(io.BytesIO(uploaded['GurNum-20230130T135920Z-001.zip']),
'r')
data.extractall()
data.printdir()
train_data_dir = 'GurNum/train'
test_data_dir = 'GurNum/val'
import os
target_names = [item for item in os.listdir(train_data_dir) if
os.path.isdir(os.path.join(train_data_dir, item))]
nb_train_samples = sum([len(files) for _, _, files in
os.walk(train_data_dir)])
nb_test_samples = sum([len(files) for _, _, files in
os.walk(test_data_dir)])
total_nb_samples = nb_train_samples + nb_test_samples
nb_classes = len(target_names)
epochs = 2
batch_size = 4
print('Training a CNN Multi-Classifier Model ......')
print('\n - names of classes: ', target_names, '\n - # of classes: ',
nb_classes)
print(' - # of trained samples: ', nb_train_samples,
'\n - # of test samples: ', nb_test_samples,
'\n - total # of samples: ', total_nb_samples,
'\n - train ratio:', round(nb_train_samples/total_nb_samples*100,
2),
'\n - test ratio:', round(nb_test_samples/total_nb_samples*100, 2),
' %', '\n - # of epochs: ', epochs, '\n - batch size: ', batch_size)
# Define the data transform to be applied on the dataset
data_transform = transforms.Compose([
transforms.Resize((32, 32)),
transforms.ToTensor(),
transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
# Load the training and validation datasets
train_dataset = torchvision.datasets.ImageFolder(train_data_dir,
transform=data_transform)
val_dataset = torchvision.datasets.ImageFolder(test_data_dir,
transform=data_transform)
# Define the data loaders for training and validation
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32,
shuffle=True)
# Define the CNN architecture
class Net(nn.Module):
def __init__(self):
super(Net, self).__init__()
self.conv1 = nn.Conv2d(3, 6, 5)
self.pool = nn.MaxPool2d(2, 2)
self.conv2 = nn.Conv2d(6, 16, 5)
self.fc1 = nn.Linear(16 * 5 * 5, 120)
self.fc2 = nn.Linear(120, 84)
self.fc3 = nn.Linear(84, nb_classes)
def forward(self, x):
x = self.pool(torch.relu(self.conv1(x)))
x = self.pool(torch.relu(self.conv2(x)))
x = x.view(-1, 16 * 5 * 5)
x = torch.relu(self.fc1(x))
x = nn.Dropout(p=0.5)(x) # Add dropout layer after fc1 layer
x = torch.relu(self.fc2(x))
x = nn.Dropout(p=0.5)(x) # Add dropout layer after fc2 layer
x = self.fc3(x)
return x
def gradient_checking(net, inputs, labels, criterion):
epsilon = 1e-6
parameters = list(net.parameters())
grads = [None] * len(parameters)
for i, p in enumerate(parameters):
p_tensor = p.data.clone()
original = p_tensor.detach().numpy()[0]
p_tensor = torch.tensor(original + epsilon,
requires_grad=True)
net.zero_grad()
output = net(inputs)
loss = criterion(output, labels)
loss.backward()
grads[i] = parameters[i].grad.detach().numpy()[0]
p_tensor = torch.tensor(original, requires_grad=True)
net.zero_grad()
output = net(inputs)
loss = criterion(output, labels)
loss.backward()
grads[i] -= parameters[i].grad.detach().numpy()[0]
grads[i] /= epsilon
return grads
# Initialize the CNN
net = Net()
# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
# Train the CNN and calculate val dataset accuracy
for epoch in range(10): # loop over the dataset multiple times
running_loss = 0.0
for i, data in enumerate(train_loader, 0):
inputs, labels = data
optimizer.zero_grad()
outputs = net(inputs)
loss = criterion(outputs, labels)
# Call the gradient checking function
net.gradient_checking(inputs, labels, criterion)
optimizer.zero_grad()
loss.backward()
optimizer.step()
running_loss += loss.item()
print('Epoch %d Train loss: %.3f' % (epoch + 1, running_loss / (i +
1)))
# Evaluate the model on the validation dataset
correct = 0
with torch.no_grad():
total = 0
for data in val_loader:
images, labels = data
outputs = net(images)
_, predicted = torch.max(outputs.data, 1)
total += labels.size(0)
correct += (predicted == labels).sum().item()
print('Accuracy of the network on the validation images: %d %%' %
(100 * correct / total))
# Define the custom L1 regularization term
def l1_regularization(model):
l1_reg = 0
for param in model.parameters():
l1_reg += torch.norm(param, p=1)
return l1_reg
# Define the custom L2 regularization term
def l2_regularization(model):
l2_reg = 0
for param in model.parameters():
l2_reg += torch.norm(param, p=2)
return l2_reg
# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
# Define the regularization strength
reg_strength = 0.1
# Define the optimizer with L1 regularization
optimizer_L1 = optim.SGD(net.parameters(), lr=0.001, momentum=0.9,
weight_decay=reg_strength)
# Train the CNN with L1 regularization
for epoch in range(10): # loop over the dataset multiple times
running_loss = 0.0
for i, data in enumerate(train_loader, 0):
inputs, labels = data
optimizer_L1.zero_grad()
outputs = net(inputs)
loss = criterion(outputs, labels) + reg_strength *
l1_regularization(net)
loss.backward()
optimizer_L1.step()
running_loss += loss.item()
print('Epoch %d loss with L1 regularization: %.3f' % (epoch + 1,
running_loss / (i + 1)))
# Define the optimizer with L2 regularization
optimizer_L2 = optim.SGD(net.parameters(), lr=0.001, momentum=0.9,
weight_decay=reg_strength)
# Train the CNN with L2 regularization
for epoch in range(10): # loop over the dataset multiple times
running_loss = 0.0
for i, data in enumerate(train_loader, 0):
inputs, labels = data
optimizer_L2.zero_grad()
outputs = net(inputs)
loss = criterion(outputs, labels) + reg_strength *
l2_regularization(net)
loss.backward()
optimizer_L2.step()
running_loss += loss.item()
print('Epoch %d loss with L2 regularization: %.3f' % (epoch + 1,
running_loss / (i + 1)))
#torch.save(net.state_dict(), 'model.pt')